# 전이 학습(Transfer Learning) - VGG16

---

# 1 침해(Dementia) 환자 뇌 영상(MRI) 분류

- [전이 학습(Transfer Learning)](https://choice-life.tistory.com/40)
- [ImageNet](https://image-net.org)
- [VGG16](https://hnsuk.tistory.com/30)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

#### 패키지 임포트

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, models, layers, optimizers, metrics
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16

import tensorflow as tf

In [ ]:
# seed 값 설정
np.random.seed(3)
tf.random.set_seed(3)

#### 데이터 로드 및 분할

In [ ]:
# Train data: Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,  #수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                   width_shift_range=0.1,  #전체 크기의 10% 범위에서 좌우로 이동합니다.
                                   height_shift_range=0.1, #마찬가지로 위, 아래로 이동합니다.
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
       './data/dementia/train',
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

In [ ]:
# Test data: No Augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
       './data/dementia/test',
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

#### VGG16 신경망 모델 불러오기

In [ ]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
transfer_model.trainable = False  # VGG16 모델의 가중치는 학습하지 않음

transfer_model.summary()

#### VGG16 신경망에 사용자 계층 추가

In [ ]:
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dense(2, activation='softmax'))

#### 모델 계층 확인

In [ ]:
finetune_model.summary()

#### 모델 컴파일

In [ ]:
finetune_model.compile(loss='sparse_categorical_crossentropy',
                       optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

#### 모델 학습 실행 및 저장

- steps_per_epoch = len(X_train)//batch_size
- validation_steps = len(X_test)//batch_size

In [ ]:
%%time
history = finetune_model.fit(train_generator, steps_per_epoch=16, epochs=20, batch_size=10,
                             validation_data=test_generator, validation_steps=12)

#### 모델 평가

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (finetune_model.evaluate(test_generator)[1]))

#### 학습 진행 과정
- history
 - loss: 훈련 손실값
 - accuracy: 훈련 정확도
 - val_loss: 검증 손실값
 - val_accuracy: 검증 정확도

In [ ]:
# 그래프로 표현
plt.plot(history.history['val_accuracy'], marker='.', c='lightcoral', label='Validation Accuracy')
plt.plot(history.history['accuracy'],     marker='.', c='red',label='Train Accuracy')

plt.plot(history.history['val_loss'], marker='.', c='cornflowerblue', label='Validation Loss')
plt.plot(history.history['loss'],     marker='.', c='blue',label='Train Loss')

plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Accuracy/Loss')
plt.legend(loc='best')

plt.title('학습 진행에 따른 학습 데이터와 검증 데이터의 정확도/에러')
plt.show()

#### 테스트 데이터(ImageDataGenerator) 에서 Image, Label 구하기

In [ ]:
# Classes: 0 = 침해(ad), 1 = 정상(normal)

test_generator.class_indices

In [ ]:
# ImageDataGenerator --> Numpy array

steps = test_generator.n // 5
images, labels = [], []

for i in range(steps):
    a, b = test_generator.next()
    images.extend(a)
    labels.extend(b)

images = np.asarray(images)
labels = np.asarray(labels)

#### 결과 예측

In [ ]:
# 예측 확률
pred_prob = finetune_model.predict(images)
pred_prob

In [ ]:
# 결과 예측
pred = np.argmax(pred_prob, axis=1).flatten()
pred

#### 결과 확인

In [ ]:
plt.imshow(images[0])
plt.title(f'Label: {pred[0]}')
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(10, 5), subplot_kw={'xticks':(), 'yticks': ()})
for i, (ax, img) in enumerate(zip(axes.ravel(), images[:10])):
    ax.imshow(img)
    ax.set_xlabel(f'Label: {pred[i]}')

#### 결과 평가

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels, pred))

---

In [ ]:
# End of file